In [ ]:
%pip install -q --upgrade google-cloud-aiplatform google-cloud-bigquery

In [ ]:
import logging
from google.cloud import bigquery
from google.cloud.aiplatform_v1beta1 import (FeaturestoreServiceClient,
                                             FeatureValueDestination,
                                             FeatureSelector,
                                             ExportFeatureValuesRequest,
                                             types)

In [ ]:
# Settings
project_id = "matt-demos"  # Change it
region_id = "us-central1" # Change it
feature_store_id = "legacy_fs_1" # Change it
entitytype_id = "benchmark_3emb_1000xfloat64" # Change it
bigquery_dest_dataset_id = "featurestore_ds" # Change it
logging.getLogger().setLevel(logging.INFO) # set to DEBUG during trobleshooting

In [ ]:
# Get vertex client (using local credentials)
fs_client = FeaturestoreServiceClient(
    client_options = {
        "api_endpoint": f"{region_id}-aiplatform.googleapis.com"
    }
)

# Get bigquery client
bq_client = bigquery.Client()

# Aux variables
parent_id = f'projects/{project_id}/locations/{region_id}'
entitytype_id_path = f'{parent_id}/featurestores/{feature_store_id}/entityTypes/{entitytype_id}'
api_endpoint = f'{region_id}-aiplatform.googleapis.com'
entity_type_short_id = entitytype_id_path.split("/")[-1]
bigquery_output_url = f'bq://{project_id}.{bigquery_dest_dataset_id}.{entity_type_short_id}'


In [ ]:
# Export EntityType to Bigquery

destination = FeatureValueDestination()
destination.bigquery_destination.output_uri = bigquery_output_url

feature_selector = FeatureSelector()
feature_selector.id_matcher.ids = ['*']

# request job
logging.info("Exporting features")
operation = fs_client.export_feature_values(
    ExportFeatureValuesRequest(
        entity_type = entitytype_id_path,
        destination = destination,
        feature_selector = feature_selector,
        full_export = types.ExportFeatureValuesRequest.FullExport()
    )
)

operation.result() # wait operation to finish

logging.info(f'Entity type "{entity_type_short_id}" from feature store \
"{feature_store_id}" located at "{region_id}" of project "{project_id}" to Bigquery table "{bigquery_output_url}"')

In [ ]:
# Show exported rows and schema
table_id = bq_client.get_table(f'{bigquery_dest_dataset_id}.{entitytype_id}')

logging.info(f'The table {table_id} has {table_id.num_rows} rows and {len(table_id.schema)} columns.\n\n Schema: {table_id.schema}')